![image.png](attachment:image.png)

##### **Good luck with taking your exam. Keep working and make your dreams all come true. Seeing the results of all of your hard work will make this struggle worth it. We’re all thinking of you.** 
<b><font color='blue'>AI-PRO Spark Team ITI</font></b>

# NLP Using PySpark

## Objective:
- The objective from this project is to create a <b>Spam filter using NaiveBayes classifier</b>.
- It is required to obtain <b>f1_scored > 0.9</b>.
- We'll use a dataset from UCI Repository. SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
- Data is also provided for you in the assignment (you do not have to download it).

## To perform this task follow the following guiding steps:

### Create a spark session and import the required libraries

In [4]:
pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Read the readme file to learn more about the data

In [29]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Fianl").getOrCreate()
df = spark.read.csv("SMSSpamCollection.txt",sep='\t')



### Read the data into a DataFrame

In [31]:
df = spark.read.load("/content/SMSSpamCollection.txt",format="csv", sep="\t", inferSchema="true")

### Print the schema

In [32]:
df.printSchema()
df.show(4)


root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
+----+--------------------+
only showing top 4 rows



### Rename the first column to 'class' and second column to 'text'

In [33]:
df = df.withColumnRenamed("_c0",'class')
df = df.withColumnRenamed("_c1", 'text')

In [34]:
df.printSchema()


root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)



### Show the first 10 rows from the dataframe
- Show once with truncate=True and once with truncate=False

In [35]:
#with truncate =True
df.show(10)


+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
+-----+--------------------+
only showing top 10 rows



In [37]:
#with truncate =False
df.show(10 , truncate=False)


+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                 |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                   |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075o

## Clean and Prepare the Data

### Create a new feature column contains the length of the text column

In [38]:
from pyspark.sql.functions import length
df = df.withColumn('length_text', length(df.text))


### Show the new dataframe

In [40]:
df.show()

+-----+--------------------+-----------+
|class|                text|length_text|
+-----+--------------------+-----------+
|  ham|Go until jurong p...|        111|
|  ham|Ok lar... Joking ...|         29|
| spam|Free entry in 2 a...|        155|
|  ham|U dun say so earl...|         49|
|  ham|Nah I don't think...|         61|
| spam|FreeMsg Hey there...|        147|
|  ham|Even my brother i...|         77|
|  ham|As per your reque...|        160|
| spam|WINNER!! As a val...|        157|
| spam|Had your mobile 1...|        154|
|  ham|I'm gonna be home...|        109|
| spam|SIX chances to wi...|        136|
| spam|URGENT! You have ...|        155|
|  ham|I've been searchi...|        196|
|  ham|I HAVE A DATE ON ...|         35|
| spam|XXXMobileMovieClu...|        149|
|  ham|Oh k...i'm watchi...|         26|
|  ham|Eh u remember how...|         81|
|  ham|Fine if thats th...|         56|
| spam|England v Macedon...|        155|
+-----+--------------------+-----------+
only showing top

### Get the average text length for each class (give alias name to the average length column)

In [44]:
df.createOrReplaceTempView("df_view")
spark.sql("""select  class , avg(length_text) 
from df_view
group By(class)
""").show()


+-----+-----------------+
|class| avg(length_text)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

### In this part you transform you raw text in to tf_idf model :
- For more information about TF-IDF check the following link: <b>(Not needed for the test)</b>
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

### Perform the following steps to obtain TF-IDF:
1. Import the required transformers/estimators for the subsequent steps.
2. Create a <b>Tokenizer</b> from the text column.
3. Create a <b>StopWordsRemover</b> to remove the <b>stop words</b> from the column obtained from the <b>Tokenizer</b>.
4. Create a <b>CountVectorizer</b> after removing the <b>stop words</b>.
5. Create the <b>TF-IDF</b> from the <b>CountVectorizer</b>.

In [56]:
#Import the required transformers/estimators for the subsequent steps.
#Create a Tokenizer from the text column.
from pyspark.ml.feature import RegexTokenizer
tokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern=" ")
tokenized = tokenizer.transform(df)
tokenized.show()

+-----+--------------------+-----------+--------------------+
|class|                text|length_text|               words|
+-----+--------------------+-----------+--------------------+
|  ham|Go until jurong p...|        111|[go, until, juron...|
|  ham|Ok lar... Joking ...|         29|[ok, lar..., joki...|
| spam|Free entry in 2 a...|        155|[free, entry, in,...|
|  ham|U dun say so earl...|         49|[u, dun, say, so,...|
|  ham|Nah I don't think...|         61|[nah, i, don't, t...|
| spam|FreeMsg Hey there...|        147|[freemsg, hey, th...|
|  ham|Even my brother i...|         77|[even, my, brothe...|
|  ham|As per your reque...|        160|[as, per, your, r...|
| spam|WINNER!! As a val...|        157|[winner!!, as, a,...|
| spam|Had your mobile 1...|        154|[had, your, mobil...|
|  ham|I'm gonna be home...|        109|[i'm, gonna, be, ...|
| spam|SIX chances to wi...|        136|[six, chances, to...|
| spam|URGENT! You have ...|        155|[urgent!, you, ha...|
|  ham|I

In [57]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover 
from nltk.stem.snowball import SnowballStemmer

#Create a StopWordsRemover to remove the stop words from the column obtained from the Tokenizer
remover = StopWordsRemover(inputCol='words', outputCol='words_clean')
df_words_no_stopw = remover.transform(tokenized)
df_words_no_stopw.show()

#Create a CountVectorizer after removing the stop words.
from pyspark.ml.feature import CountVectorizer
import pyspark.ml.feature
cvec = CountVectorizer(inputCol = "words_clean" , outputCol="features")
model = cvec.fit(df_words_no_stopw)

#Create the TF-IDF from the CountVectorizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
idf = IDF(inputCol="features", outputCol="tf_idf")




+-----+--------------------+-----------+--------------------+--------------------+
|class|                text|length_text|               words|         words_clean|
+-----+--------------------+-----------+--------------------+--------------------+
|  ham|Go until jurong p...|        111|[go, until, juron...|[go, jurong, poin...|
|  ham|Ok lar... Joking ...|         29|[ok, lar..., joki...|[ok, lar..., joki...|
| spam|Free entry in 2 a...|        155|[free, entry, in,...|[free, entry, 2, ...|
|  ham|U dun say so earl...|         49|[u, dun, say, so,...|[u, dun, say, ear...|
|  ham|Nah I don't think...|         61|[nah, i, don't, t...|[nah, think, goes...|
| spam|FreeMsg Hey there...|        147|[freemsg, hey, th...|[freemsg, hey, da...|
|  ham|Even my brother i...|         77|[even, my, brothe...|[even, brother, l...|
|  ham|As per your reque...|        160|[as, per, your, r...|[per, request, 'm...|
| spam|WINNER!! As a val...|        157|[winner!!, as, a,...|[winner!!, valued...|
| sp

- Convert the <b>class column</b> to index using <b>StringIndexer</b>
- Create feature column from the <b>TF-IDF</b> and <b>lenght</b> columns.

In [58]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder ,VectorAssembler

ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')
clean_up = VectorAssembler(inputCols=['tf_idf','length_text'],outputCol='feature')

## The Model
- Create a <b>NaiveBayes</b> classifier with the default parameters.

In [59]:
from pyspark.ml.classification import NaiveBayes
# Use defaults
nb = NaiveBayes()

## Pipeline
### Create a pipeline model contains all the steps starting from the Tokenizer to the NaiveBays classifier.

In [60]:
from pyspark.ml import Pipeline
data_prep_pipe = Pipeline(stages=[ham_spam_to_num , tokenizer , remover , model , idf , clean_up])
cleaner = data_prep_pipe.fit(df)

In [62]:
clean_data = cleaner.transform(df)
clean_data = clean_data.select(['label','features'])
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13422,[6,10,30,6...|
|  0.0|(13422,[0,23,296,...|
|  1.0|(13422,[2,12,18,2...|
|  0.0|(13422,[0,69,79,1...|
|  0.0|(13422,[35,131,31...|
|  1.0|(13422,[9,59,139,...|
|  0.0|(13422,[9,52,103,...|
|  0.0|(13422,[124,184,4...|
|  1.0|(13422,[1,46,118,...|
|  1.0|(13422,[0,1,12,26...|
|  0.0|(13422,[17,42,120...|
|  1.0|(13422,[7,15,36,8...|
|  1.0|(13422,[12,29,46,...|
|  0.0|(13422,[38,95,216...|
|  0.0|(13422,[549,1694,...|
|  1.0|(13422,[29,107,10...|
|  0.0|(13422,[81,211,47...|
|  0.0|(13422,[0,2,48,13...|
|  0.0|(13422,[0,73,101,...|
|  1.0|(13422,[3,29,32,5...|
+-----+--------------------+
only showing top 20 rows



### Split your data to trian and test data with ratios 0.7 and 0.3 respectively.

In [63]:
(training,testing) = clean_data.randomSplit([0.7,0.3])


### Fit your Pipeline model to the training data

In [64]:
spam_predictor = nb.fit(training)


### Perform predictions on tests dataframe

In [65]:
test_results = spam_predictor.transform(testing)
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13422,[0,1,3,49,...|[-121.95557663872...|[0.99999998339641...|       0.0|
|  0.0|(13422,[0,1,8,13,...|[-95.539114081548...|[0.99999999999732...|       0.0|
|  0.0|(13422,[0,1,8,13,...|[-95.539114081548...|[0.99999999999732...|       0.0|
|  0.0|(13422,[0,1,10,31...|[-140.04660237119...|[0.99999999996161...|       0.0|
|  0.0|(13422,[0,1,14,19...|[-114.43959350143...|[0.99999999992814...|       0.0|
|  0.0|(13422,[0,1,15,18...|[-120.20830803274...|[0.99999493510126...|       0.0|
|  0.0|(13422,[0,1,17,20...|[-117.68543274588...|[0.99999911306138...|       0.0|
|  0.0|(13422,[0,1,23,30...|[-59.870479281761...|[0.99999999699193...|       0.0|
|  0.0|(13422,[0,1,26,34...|[-193.12945030878...|[0.99999987879348...|       0.0|
|  0.0|(13422,[0

### Print the schema of the prediction dataframe

In [66]:
test_results.printSchema()


root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



## Model Evaluation
- Use <b>MulticlassClassificationEvaluator</b> to calculate the <b>f1_score</b>.

In [68]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model during predicting spam was: {}".format(acc))

Accuracy of model during predicting spam was: 0.9661277058128581


f1_score is: 0.9664707489549014


# GOOD LUCK
<b><font color='GREEN'>AI-PRO Spark Team ITI</font></b>

![image-3.png](attachment:image-3.png)